<a href="https://colab.research.google.com/github/LinusBach/SentimentAnalysis/blob/main/sentiAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple sentiment analysis

Sentiment analysis, using iMDB database

First, implement and train a feedforward NN model with TF-IDF. And then train your
model using word2vec embedding. Report both training and development accuracy on
the dataset. Try to use stochastic gradient descent or (mini-batch) stochastic gradient
descent!

requires word2vec model https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing

In [1]:
!pip install tensorflow
!pip install nltk
!pip install gensim

### imports


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras
import nltk
import csv
import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
import gensim

# minimum ocuurences for a word to be regarded
VOCAB_SIZE = 5000
# number of most frequent words to be disregarded
HIGHER_CUTOFF = 0

### load dataset into memory
return a list of docs and a list of respective labels

In [3]:
def load_data (filename):
  content = list()
  labels = list()

  has_header = True
  # detect if file has a header
  # with open(filename, 'r') as file:
  #   sample = file.read(64)
  #   has_header = csv.Sniffer().has_header(sample)

  with open(filename, 'r') as file:
    reader = csv.reader(file)
    # skip first line if file has a header 
    if has_header:
      next(reader)
    for c, l in reader:
      content.append(c)
      labels.append(l)
  return content, labels

### turn a dataset into clean tokens

In [4]:
def clean_data(data):
  corpus = list()
  corp_voc = dict()
  stop = nltk.corpus.stopwords.words("english")
  # regex tokenizer, find words, numbers and words containing '
  tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+(?:'\w)?")
  for doc in data:
    doc = tokenizer.tokenize(doc)
    doc_cleaned = dict()
    for tok in doc:
      # make all words lower case
      tok = tok.lower()
      # filter out numbers 
      if not tok.isdigit() and tok not in stop:
        # add clean token to document
        if tok in doc_cleaned:
          doc_cleaned[tok] += 1
        else:
          doc_cleaned[tok] = 1
    for tok in doc_cleaned.keys():
    # increase corpus vocabulary
        if tok in corp_voc:
          corp_voc[tok] += 1
        else:
          corp_voc[tok] = 1
    corpus.append(doc_cleaned)
  return corpus, corp_voc

# filter all words out of a corpus that are not in a vocabulary
def get_filtered_corpus(corpus, vocab):
  clean_corpus = list()
  for doc in corpus:
    clean_doc = dict()
    for tok in doc:
      if tok in vocab:
        clean_doc[tok] = doc[tok]
    clean_corpus.append(clean_doc)
  return clean_corpus

### preprocess the dataset

tf-idf vectorizer

In [5]:

# corpus must be a list of dicts of form (token: occurences)
# vocab must be a dict of form (token: documents in corpus containing token)
def preprocess_tf_idf(corpus, vocab):
  processed = np.zeros((len(corpus), len(vocab)))
  idf = get_idf(vocab)
  token_order = {tok: i for i, tok, in enumerate(sorted(vocab.keys()))}
  for n_doc, doc in enumerate(corpus):
    tf = get_tf(doc)
    for tok in set(doc):
      tok_pos = token_order[tok]
      processed[n_doc][tok_pos] = tf[tok]*idf[tok]
  return processed

def get_tf(doc):
  tf = dict()
  for tok, occ in doc.items():
    tf[tok] = occ / len(doc)
  return tf

def get_idf(corp_voc):
  idf = dict()
  for tok, docs_containing in corp_voc.items():
    idf[tok] = np.log10(len(corp_voc) / docs_containing)
  return idf


semantic embeddings
pooling word2vec embeddings

In [45]:
def preprocess_sem_vec(corpus, vocab_vectors):
  processed = list()
  for n_doc, doc in enumerate(corpus):
    vecs = [occ*vocab_vectors[tok] for tok, occ in doc.items() if tok in vocab_vectors]
    processed.append(np.mean(vecs, axis=0))
  return np.array(processed)

### define the model


In [20]:
def define_model(input_dim):
  model = keras.models.Sequential()
  model.add(layers.Dense(511, activation='relu'))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(255, activation='relu'))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(127, activation='relu'))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(63, activation='relu'))
  model.add(layers.Dense(2, activation='softmax'))
  return model

### classify a review as negative or positive.

In [21]:
def predict_sentiment(model, doc):
  return model.predict(doc)

## Running the model

Setting constants

In [22]:
LEARNING_RATE = 0.001
BATCH_SIZE = 32
EPOCHS = 20

load training data

In [23]:
# read data from file
raw_data, labels = load_data("Train.csv")
full_corpus, full_vocab = clean_data(raw_data)
# select part of vocabulary
frequencies = sorted(full_vocab.items(), key=lambda x : x[1], reverse=True)
vocab = {x[0] : x[1] for x in frequencies[HIGHER_CUTOFF:HIGHER_CUTOFF+VOCAB_SIZE]}
corpus = get_filtered_corpus(full_corpus, vocab.keys())
# process labels
one_hot_labels = keras.utils.to_categorical(labels)

load validation data

In [24]:
v_raw, v_labels = load_data("Valid.csv")
full_valid_corpus, _ = clean_data(v_raw)
valid_corpus = get_filtered_corpus(full_valid_corpus, vocab)
valid_labels = keras.utils.to_categorical(v_labels)

load testing data

In [25]:
t_raw, t_labels = load_data("Test.csv")
full_test_corpus, _ = clean_data(t_raw)
test_corpus = get_filtered_corpus(full_test_corpus, vocab)
test_labels = keras.utils.to_categorical(t_labels)

train a model on tf-idf data 

In [26]:
# get tf-idf representation of data
tfidf_data = preprocess_tf_idf(corpus, vocab)
test_data = preprocess_tf_idf(test_corpus, vocab)
valid_data = preprocess_tf_idf(valid_corpus, vocab)

In [27]:
# build model
tfidf_model = define_model(VOCAB_SIZE,)
tfidf_model.compile(
    loss='mse',
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
tfidf_model.fit(
    tfidf_data,
    one_hot_labels,
    validation_data=(valid_data, valid_labels),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)
print("begin evaluation")
tfidf_model.evaluate(test_data, test_labels)

begin training
Epoch 1/20
1250/1250 [==============================] - 6s 5ms/step - loss: 0.1124 - accuracy: 0.8376 - val_loss: 0.0940 - val_accuracy: 0.8716
Epoch 2/20
1250/1250 [==============================] - 6s 4ms/step - loss: 0.0738 - accuracy: 0.9014 - val_loss: 0.0902 - val_accuracy: 0.8748
Epoch 3/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0623 - accuracy: 0.9201 - val_loss: 0.0925 - val_accuracy: 0.8784
Epoch 4/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0494 - accuracy: 0.9397 - val_loss: 0.0941 - val_accuracy: 0.8766
Epoch 5/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0367 - accuracy: 0.9564 - val_loss: 0.0998 - val_accuracy: 0.8760
Epoch 6/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0290 - accuracy: 0.9668 - val_loss: 0.1113 - val_accuracy: 0.8678
Epoch 7/20
1250/1250 [==============================] - 5s 4ms/step - loss: 0.0236 - accuracy: 0.9736 - val_loss: 0.1057 

[0.11472126096487045, 0.871999979019165]

train a model on word2vec embeddings 

In [46]:
# get semantic embeddings of data
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
vocab_vecs = {tok : word2vec_model[tok] for tok in vocab.keys() if tok in word2vec_model.key_to_index}
w2v_train_data = preprocess_sem_vec(corpus, vocab_vecs)
w2v_valid_data = preprocess_sem_vec(valid_corpus, vocab_vecs)
w2v_test_data = preprocess_sem_vec(test_corpus, vocab_vecs)

In [47]:
# build model
w2v_model = define_model(300,)
w2v_model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy'])
# train model
print("begin training")
w2v_model.fit(
    w2v_train_data,
    one_hot_labels,
    validation_data=(w2v_valid_data, valid_labels),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE)
print("begin evaluation")
w2v_model.evaluate(w2v_test_data, test_labels)

begin training
Epoch 1/20
1250/1250 [==============================] - 3s 2ms/step - loss: 0.4072 - accuracy: 0.8143 - val_loss: 0.3458 - val_accuracy: 0.8548
Epoch 2/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3618 - accuracy: 0.8428 - val_loss: 0.3386 - val_accuracy: 0.8592
Epoch 3/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3517 - accuracy: 0.8472 - val_loss: 0.3332 - val_accuracy: 0.8598
Epoch 4/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3482 - accuracy: 0.8489 - val_loss: 0.3295 - val_accuracy: 0.8600
Epoch 5/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3436 - accuracy: 0.8513 - val_loss: 0.3329 - val_accuracy: 0.8554
Epoch 6/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3392 - accuracy: 0.8527 - val_loss: 0.3329 - val_accuracy: 0.8648
Epoch 7/20
1250/1250 [==============================] - 2s 2ms/step - loss: 0.3360 - accuracy: 0.8540 - val_loss: 0.3287 

[0.3334038555622101, 0.8611999750137329]